# Census Datasets example

*Goal:* demonstrate basic use of the `census_datasets` dataframe.

Each Cell Census contains a top-level dataframe itemizing the datasets contained therein. You can read this into a Pandas DataFrame:

In [1]:
import cell_census
import tiledbsoma as soma

census = cell_census.open_soma()
census_datasets = census["census_info"]["datasets"].read().concat().to_pandas()

# for convenience, indexing on the soma_joinid which links this to other census data.
census_datasets = census_datasets.set_index("soma_joinid")
census_datasets

,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,
0,,,,d2fc9880-e6d3-4922-af5c-61f4f517adfa,,d2fc9880-e6d3-4922-af5c-61f4f517adfa.h5ad,99369
1,,,,f75f2ff4-2884-4c2d-b375-70de37a34507,,f75f2ff4-2884-4c2d-b375-70de37a34507.h5ad,3799
2,,,,703f00e6-b996-48e5-bc34-00c41b9876f4,,703f00e6-b996-48e5-bc34-00c41b9876f4.h5ad,649
3,,,,cdefb878-7f00-4b9d-9eda-b3652cfac0c8,,cdefb878-7f00-4b9d-9eda-b3652cfac0c8.h5ad,1641
4,,,,fd072bc3-2dfb-46f8-b4e3-467cb3223182,,fd072bc3-2dfb-46f8-b4e3-467cb3223182.h5ad,908046
...,...,...,...,...,...,...,...
469,,,,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf,,2adb1f8a-a6b1-4909-8ee8-484814e2d4bf.h5ad,598266
470,,,,2190bd4d-3be0-4bf7-8ca8-8d6f71228936,,2190bd4d-3be0-4bf7-8ca8-8d6f71228936.h5ad,126782
471,,,,1a018108-b4b6-457b-ba15-046d5e98c169,,1a018108-b4b6-457b-ba15-046d5e98c169.h5ad,21534


The sum cells across all datasets should match the number of cells across all SOMA experiments (human, mouse).

In [2]:
# Count cells across all experiments
all_experiments = (
    (organism_name, organism_experiment) for organism_name, organism_experiment in census["census_data"].items()
)
experiments_total_cells = 0
print("Count by experiment:")
for organism_name, organism_experiment in all_experiments:
    num_cells = len(organism_experiment.obs.read(column_names=["soma_joinid"]).concat().to_pandas())
    print(f"\t{num_cells} cells in {organism_name}")
    experiments_total_cells += num_cells

print(f"\nFound {experiments_total_cells} cells in all experiments.")

# Count cells across all datasets
print(f"Found {census_datasets.dataset_total_cell_count.sum()} cells in all datasets.")

Count by experiment:
	43246174 cells in homo_sapiens
	3922090 cells in mus_musculus

Found 47168264 cells in all experiments.
Found 47168264 cells in all datasets.


Lets pick one dataset to slice out of the census, and turn into an [AnnData](https://anndata.readthedocs.io/en/latest/) in-memory object. This can be used with the [ScanPy](https://scanpy.readthedocs.io/en/stable/) toolchain. You can also save this AnnData locally using the AnnData [`write`](https://anndata.readthedocs.io/en/latest/api.html#writing) API.

In [3]:
census_datasets[census_datasets.dataset_id == "0bd1a1de-3aee-40e0-b2ec-86c7a30c7149"]

,collection_id,collection_name,collection_doi,dataset_id,dataset_title,dataset_h5ad_path,dataset_total_cell_count
soma_joinid,,,,,,,
406,,,,0bd1a1de-3aee-40e0-b2ec-86c7a30c7149,,0bd1a1de-3aee-40e0-b2ec-86c7a30c7149.h5ad,40220


Create a query on the mouse experiment, "RNA" measurement, for the dataset_id.

In [4]:
mouse = census["census_data"]["mus_musculus"]
with mouse.axis_query(
    "RNA",
    obs_query=soma.AxisQuery(value_filter="dataset_id == '0bd1a1de-3aee-40e0-b2ec-86c7a30c7149'"),
) as query:
    adata = query.to_anndata("raw")

adata

AnnData object with n_obs × n_vars = 40220 × 52392
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_general', 'tissue_general_ontology_term_id'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length'

You can also use the `cell_census.get_h5ad_uri()` API to fetch a URI pointing to the H5AD associated with this `dataset_id`. This is the same H5AD you can download from the CELLxGENE Portal, and may contain additional data-submittor provided information which was not included in the Cell Census.

The "locator" returned by this API will include a `uri` and additional information that may be necessary to use the URI (eg, the S3 region).

You will need to use a download API to fetch this H5AD, such as [`fsspec`](https://filesystem-spec.readthedocs.io/en/latest/).

In [5]:
cell_census.get_source_h5ad_uri("0bd1a1de-3aee-40e0-b2ec-86c7a30c7149")

{'uri': 's3://cellxgene-data-public/cell-census/2023-01-30/h5ads/0bd1a1de-3aee-40e0-b2ec-86c7a30c7149.h5ad',
 's3_region': 'us-west-2'}